# TrustHire – AI-Powered Job Scam Detection System

## Machine Learning Pipeline for Detecting Fake Job Offers

This notebook implements a complete ML pipeline for:
1. **Text Classification** - Classify job offers as Scam/Genuine
2. **NLP Feature Extraction** - Extract keywords, sentiment, and red flags
3. **Image OCR + Analysis** - Extract and analyze text from screenshots

---

## 1. Setup and Dependencies

In [ ]:
# Install required packages (run once)
!pip install pandas numpy scikit-learn nltk spacy pytesseract pillow transformers torch joblib matplotlib seaborn wordcloud

In [ ]:
# Core imports
import pandas as pd
import numpy as np
import re
import json
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.pipeline import Pipeline

# NLP libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')

# OCR
from PIL import Image
import pytesseract

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Model persistence
import joblib

print("✅ All libraries imported successfully!")

## 2. Dataset Preparation

We'll create a synthetic dataset of job postings labeled as scam or genuine.
In production, you would use real labeled data.

In [ ]:
# Synthetic dataset for training (In production, use real labeled data)
# Label: 1 = Scam, 0 = Genuine

scam_examples = [
    "Congratulations! You have been selected for a high-paying job. Pay $200 registration fee to proceed.",
    "Urgent hiring! Work from home and earn $5000/week. No experience needed. Send bank details.",
    "Dear candidate, you are shortlisted. Transfer Rs 5000 for training materials. Contact on WhatsApp.",
    "Immediate joining! $100/hour data entry job. Pay security deposit of $500.",
    "You won our job lottery! Send personal documents and bank account to claim position.",
    "Limited time offer! High salary job guaranteed. Registration fee required. Act now!",
    "Hiring for multinational company. 50K salary. Pay processing fee. WhatsApp only.",
    "Work from home earning $10000 monthly. Just need your SSN and bank routing number.",
    "Congratulation! Selected for interview. Pay $100 for background verification.",
    "Job offer from Google! Salary $200k. Send money for visa processing.",
    "Urgent requirement! Pay Rs 2000 for id card and uniform. Join tomorrow!",
    "You are hired! Send copy of passport and $300 for equipment deposit.",
    "Dream job opportunity! No interview needed. Just pay joining fee of $150.",
    "Selected for Amazon position. Pay $400 for training. Contact: scammer@gmail.com",
    "High paying job! $500/day guaranteed. Send bank details for salary setup.",
    "Congratulations on selection! Pay advance salary tax of Rs 10000.",
    "Work from home data entry. $50/hour. Pay $200 for software license.",
    "Immediate hiring! Send OTP received on your phone to verify account.",
    "Job confirmed at Microsoft! Transfer $600 for work visa.",
    "Freelance opportunity! $1000/week. Need your credit card for verification."
]

genuine_examples = [
    "We are pleased to offer you the position of Software Engineer at our company with annual salary of $85,000.",
    "Thank you for interviewing. We would like to invite you for a second round at our office.",
    "Your application for Data Analyst has been received. HR will contact you within 5 business days.",
    "We are looking for experienced Python developers. Apply through our careers portal.",
    "Interview scheduled for Monday 10 AM at our headquarters. Please bring your resume and ID.",
    "Position: Marketing Manager. Salary: $70,000 + benefits. Apply at careers.company.com",
    "Thank you for applying. Your resume has been forwarded to the hiring manager.",
    "We are hiring interns for our summer program. Stipend: $1500/month. Apply online.",
    "Job offer attached. Please review the terms and sign the acceptance letter.",
    "Congratulations on passing the technical interview. Final HR round scheduled for Friday.",
    "We require 3+ years experience in React.js. Competitive salary and remote options.",
    "Your background check is complete. Onboarding documents will be sent via official email.",
    "Position filled. Thank you for your interest. We will keep your resume on file.",
    "Salary negotiation meeting scheduled. Please confirm your availability.",
    "Job opening: Senior Developer at ABC Corp. Apply through LinkedIn or company website.",
    "HR department will send you the offer letter. No fees or deposits required.",
    "Welcome aboard! Your first day is March 15. Report to HR for orientation.",
    "We offer health insurance, 401k, and 20 days PTO. Review benefits package attached.",
    "Technical assessment results: Passed. Next step is the team fit interview.",
    "Reference check completed successfully. Offer letter will be emailed today."
]

# Create DataFrame
data = {
    'text': scam_examples + genuine_examples,
    'label': [1] * len(scam_examples) + [0] * len(genuine_examples)
}

df = pd.DataFrame(data)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle

print(f"Dataset shape: {df.shape}")
print(f"\nClass distribution:")
print(df['label'].value_counts())
df.head(10)

## 3. Text Preprocessing

In [ ]:
class TextPreprocessor:
    """
    Text preprocessing pipeline for job scam detection.
    Handles cleaning, tokenization, and lemmatization.
    """
    
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
        # Keep important negation words
        self.stop_words -= {'no', 'not', 'don', "don't", 'won', "won't", 'shouldn', "shouldn't"}
    
    def clean_text(self, text):
        """Basic text cleaning"""
        if not isinstance(text, str):
            return ""
        
        # Convert to lowercase
        text = text.lower()
        
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        
        # Remove email addresses but keep a marker
        text = re.sub(r'\S+@\S+', ' EMAIL_ADDRESS ', text)
        
        # Detect money amounts
        text = re.sub(r'\$[\d,]+|rs\.?\s*[\d,]+|₹[\d,]+', ' MONEY_AMOUNT ', text, flags=re.IGNORECASE)
        
        # Remove special characters but keep spaces
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    def tokenize_and_lemmatize(self, text):
        """Tokenize and lemmatize text"""
        tokens = word_tokenize(text)
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens 
                  if token not in self.stop_words and len(token) > 2]
        return ' '.join(tokens)
    
    def preprocess(self, text):
        """Full preprocessing pipeline"""
        cleaned = self.clean_text(text)
        processed = self.tokenize_and_lemmatize(cleaned)
        return processed

# Initialize preprocessor
preprocessor = TextPreprocessor()

# Apply preprocessing to dataset
df['processed_text'] = df['text'].apply(preprocessor.preprocess)

print("Sample preprocessed texts:")
for i in range(3):
    print(f"\nOriginal: {df.iloc[i]['text'][:80]}...")
    print(f"Processed: {df.iloc[i]['processed_text'][:80]}...")

## 4. Feature Engineering - Scam Indicators

In [ ]:
class ScamFeatureExtractor:
    """
    Extract scam-specific features from job postings.
    """
    
    def __init__(self):
        self.sia = SentimentIntensityAnalyzer()
        
        # Scam indicator keywords
        self.payment_words = {'pay', 'fee', 'deposit', 'transfer', 'money', 'payment', 
                              'registration', 'processing', 'advance', 'cost', 'charge'}
        self.urgency_words = {'urgent', 'immediately', 'now', 'today', 'asap', 'hurry',
                              'limited', 'act', 'quick', 'fast', 'deadline'}
        self.too_good_words = {'guaranteed', 'easy', 'simple', 'no experience', 'dream',
                               'amazing', 'incredible', 'lottery', 'won', 'selected'}
        self.personal_info_words = {'bank', 'account', 'ssn', 'passport', 'id', 'credit card',
                                    'routing', 'password', 'otp', 'verification'}
        self.suspicious_contact = {'whatsapp', 'telegram', 'gmail', 'yahoo', 'hotmail'}
    
    def count_keyword_matches(self, text, keywords):
        """Count how many keywords from a set appear in text"""
        text_lower = text.lower()
        return sum(1 for word in keywords if word in text_lower)
    
    def extract_features(self, text):
        """Extract all scam-related features from text"""
        text_lower = text.lower()
        
        features = {
            # Keyword-based features
            'payment_mentions': self.count_keyword_matches(text, self.payment_words),
            'urgency_mentions': self.count_keyword_matches(text, self.urgency_words),
            'too_good_mentions': self.count_keyword_matches(text, self.too_good_words),
            'personal_info_requests': self.count_keyword_matches(text, self.personal_info_words),
            'suspicious_contact': self.count_keyword_matches(text, self.suspicious_contact),
            
            # Pattern-based features
            'has_money_amount': 1 if re.search(r'\$[\d,]+|rs\.?\s*[\d,]+|₹[\d,]+', text_lower) else 0,
            'exclamation_count': text.count('!'),
            'caps_ratio': sum(1 for c in text if c.isupper()) / max(len(text), 1),
            'word_count': len(text.split()),
            
            # Sentiment features
            'sentiment_compound': self.sia.polarity_scores(text)['compound'],
            'sentiment_positive': self.sia.polarity_scores(text)['pos'],
            'sentiment_negative': self.sia.polarity_scores(text)['neg'],
            
            # Red flag combinations
            'payment_urgency_combo': 1 if (self.count_keyword_matches(text, self.payment_words) > 0 
                                           and self.count_keyword_matches(text, self.urgency_words) > 0) else 0,
        }
        
        return features
    
    def get_red_flags(self, text):
        """Get list of detected red flags in text"""
        red_flags = []
        text_lower = text.lower()
        
        if self.count_keyword_matches(text, self.payment_words) > 0:
            red_flags.append("Payment or fee requested")
        if self.count_keyword_matches(text, self.urgency_words) > 0:
            red_flags.append("Urgency tactics detected")
        if self.count_keyword_matches(text, self.too_good_words) > 0:
            red_flags.append("Unrealistic promises made")
        if self.count_keyword_matches(text, self.personal_info_words) > 0:
            red_flags.append("Personal/banking information requested")
        if self.count_keyword_matches(text, self.suspicious_contact) > 0:
            red_flags.append("Unprofessional contact methods")
        if re.search(r'\$[\d,]+|rs\.?\s*[\d,]+', text_lower):
            red_flags.append("Specific money amount mentioned")
        if text.count('!') >= 2:
            red_flags.append("Excessive exclamation marks")
            
        return red_flags

# Initialize feature extractor
feature_extractor = ScamFeatureExtractor()

# Extract features for all samples
feature_list = df['text'].apply(feature_extractor.extract_features).tolist()
feature_df = pd.DataFrame(feature_list)

print("Extracted Features:")
feature_df.head()

## 5. Model Training and Evaluation

In [ ]:
# Prepare data
X_text = df['processed_text']
X_features = feature_df
y = df['label']

# Split data
X_text_train, X_text_test, X_feat_train, X_feat_test, y_train, y_test = train_test_split(
    X_text, X_features, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {len(X_text_train)}")
print(f"Test set size: {len(X_text_test)}")

In [ ]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(
    max_features=500,
    ngram_range=(1, 2),  # Unigrams and bigrams
    min_df=1,
    max_df=0.95
)

# Fit and transform training data
X_tfidf_train = tfidf_vectorizer.fit_transform(X_text_train)
X_tfidf_test = tfidf_vectorizer.transform(X_text_test)

# Combine TF-IDF features with custom features
X_combined_train = np.hstack([X_tfidf_train.toarray(), X_feat_train.values])
X_combined_test = np.hstack([X_tfidf_test.toarray(), X_feat_test.values])

print(f"TF-IDF features: {X_tfidf_train.shape[1]}")
print(f"Custom features: {X_feat_train.shape[1]}")
print(f"Total features: {X_combined_train.shape[1]}")

In [ ]:
# Train multiple models and compare
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42)
}

results = []

for name, model in models.items():
    # Train model
    if name == 'Naive Bayes':
        # Naive Bayes needs non-negative features
        model.fit(X_tfidf_train, y_train)
        y_pred = model.predict(X_tfidf_test)
        y_prob = model.predict_proba(X_tfidf_test)[:, 1]
    else:
        model.fit(X_combined_train, y_train)
        y_pred = model.predict(X_combined_test)
        y_prob = model.predict_proba(X_combined_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'ROC-AUC': roc_auc
    })
    
    print(f"\n{'='*50}")
    print(f"{name}")
    print(f"{'='*50}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Genuine', 'Scam']))

# Results comparison
results_df = pd.DataFrame(results).sort_values('Accuracy', ascending=False)
print("\n" + "="*50)
print("MODEL COMPARISON")
print("="*50)
print(results_df.to_string(index=False))

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
colors = sns.color_palette("husl", len(results_df))
axes[0].barh(results_df['Model'], results_df['Accuracy'], color=colors)
axes[0].set_xlabel('Accuracy')
axes[0].set_title('Model Accuracy Comparison')
axes[0].set_xlim([0, 1])
for i, v in enumerate(results_df['Accuracy']):
    axes[0].text(v + 0.01, i, f'{v:.3f}', va='center')

# ROC-AUC comparison
axes[1].barh(results_df['Model'], results_df['ROC-AUC'], color=colors)
axes[1].set_xlabel('ROC-AUC Score')
axes[1].set_title('Model ROC-AUC Comparison')
axes[1].set_xlim([0, 1])
for i, v in enumerate(results_df['ROC-AUC']):
    axes[1].text(v + 0.01, i, f'{v:.3f}', va='center')

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Best Model - Random Forest Analysis

In [ ]:
# Train final Random Forest model
best_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

best_model.fit(X_combined_train, y_train)

# Feature importance
feature_names = list(tfidf_vectorizer.get_feature_names_out()) + list(feature_df.columns)
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 20 features
plt.figure(figsize=(10, 8))
top_features = importance_df.head(20)
sns.barplot(data=top_features, x='importance', y='feature', palette='viridis')
plt.title('Top 20 Most Important Features for Scam Detection')
plt.xlabel('Feature Importance')
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nTop 10 Important Features:")
print(importance_df.head(10).to_string(index=False))

In [ ]:
# Confusion Matrix
y_pred_best = best_model.predict(X_combined_test)

plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Genuine', 'Scam'], 
            yticklabels=['Genuine', 'Scam'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Random Forest Model')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. OCR - Image Text Extraction

In [ ]:
class ImageOCR:
    """
    Extract text from images (screenshots, job postings, chat messages).
    Uses Tesseract OCR.
    """
    
    def __init__(self):
        # Configure Tesseract path if needed (Windows)
        # pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        pass
    
    def preprocess_image(self, image):
        """Preprocess image for better OCR results"""
        from PIL import ImageFilter, ImageEnhance
        
        # Convert to grayscale
        if image.mode != 'L':
            image = image.convert('L')
        
        # Enhance contrast
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(2)
        
        # Sharpen
        image = image.filter(ImageFilter.SHARPEN)
        
        return image
    
    def extract_text(self, image_path):
        """Extract text from an image file"""
        try:
            image = Image.open(image_path)
            processed = self.preprocess_image(image)
            text = pytesseract.image_to_string(processed)
            return text.strip()
        except Exception as e:
            return f"Error extracting text: {str(e)}"
    
    def extract_text_from_bytes(self, image_bytes):
        """Extract text from image bytes (for web uploads)"""
        from io import BytesIO
        try:
            image = Image.open(BytesIO(image_bytes))
            processed = self.preprocess_image(image)
            text = pytesseract.image_to_string(processed)
            return text.strip()
        except Exception as e:
            return f"Error extracting text: {str(e)}"

# Initialize OCR
ocr = ImageOCR()

# Example usage (uncomment with actual image path)
# extracted_text = ocr.extract_text('path/to/job_posting_screenshot.png')
# print("Extracted Text:")
# print(extracted_text)

print("✅ OCR module ready!")
print("Note: Install Tesseract OCR on your system for image processing.")
print("Ubuntu: sudo apt-get install tesseract-ocr")
print("Windows: Download from https://github.com/UB-Mannheim/tesseract/wiki")

## 8. Complete TrustHire Analyzer Class

In [ ]:
class TrustHireAnalyzer:
    """
    Complete Job Scam Detection System.
    Combines ML classification, NLP feature extraction, and OCR.
    """
    
    def __init__(self, model, vectorizer, feature_extractor, preprocessor, ocr):
        self.model = model
        self.vectorizer = vectorizer
        self.feature_extractor = feature_extractor
        self.preprocessor = preprocessor
        self.ocr = ocr
    
    def analyze_text(self, text):
        """
        Analyze a job offer text and return detailed results.
        """
        # Preprocess
        processed_text = self.preprocessor.preprocess(text)
        
        # Extract features
        custom_features = self.feature_extractor.extract_features(text)
        
        # Vectorize
        tfidf_features = self.vectorizer.transform([processed_text])
        
        # Combine features
        combined_features = np.hstack([tfidf_features.toarray(), 
                                       np.array(list(custom_features.values())).reshape(1, -1)])
        
        # Predict
        prediction = self.model.predict(combined_features)[0]
        probability = self.model.predict_proba(combined_features)[0]
        
        # Calculate trust score (inverse of scam probability)
        scam_probability = probability[1]
        trust_score = int((1 - scam_probability) * 100)
        
        # Determine status
        if trust_score >= 80:
            status = "genuine"
            status_label = "Likely Genuine ✓"
        elif trust_score >= 50:
            status = "caution"
            status_label = "Proceed with Caution ⚠"
        else:
            status = "scam"
            status_label = "Likely Scam ⚠"
        
        # Get red flags
        red_flags = self.feature_extractor.get_red_flags(text)
        
        # Generate advice
        advice = self._generate_advice(status, red_flags)
        
        return {
            'trust_score': trust_score,
            'status': status,
            'status_label': status_label,
            'scam_probability': round(scam_probability * 100, 2),
            'red_flags': red_flags,
            'feature_analysis': custom_features,
            'advice': advice
        }
    
    def analyze_image(self, image_path):
        """
        Analyze an image (screenshot) by extracting text first.
        """
        extracted_text = self.ocr.extract_text(image_path)
        
        if extracted_text.startswith("Error"):
            return {
                'error': extracted_text,
                'trust_score': 0,
                'status': 'error'
            }
        
        result = self.analyze_text(extracted_text)
        result['extracted_text'] = extracted_text
        return result
    
    def _generate_advice(self, status, red_flags):
        """Generate safety advice based on analysis"""
        advice = []
        
        if status == "scam":
            advice.append("⚠️ HIGH ALERT: Do not proceed with this offer")
            advice.append("Report this to cyber crime authorities")
        elif status == "caution":
            advice.append("Proceed with caution - verify all details")
        else:
            advice.append("This offer appears legitimate, but stay vigilant")
        
        # Add specific advice based on red flags
        if "Payment or fee requested" in red_flags:
            advice.append("Never pay money to get a job")
        if "Personal/banking information requested" in red_flags:
            advice.append("Don't share bank details before official onboarding")
        if "Unprofessional contact methods" in red_flags:
            advice.append("Verify contact through official company channels")
        
        advice.append("Cross-verify company on official registries")
        advice.append("Check LinkedIn for employee reviews")
        
        return advice

# Create the analyzer instance
analyzer = TrustHireAnalyzer(
    model=best_model,
    vectorizer=tfidf_vectorizer,
    feature_extractor=feature_extractor,
    preprocessor=preprocessor,
    ocr=ocr
)

print("✅ TrustHire Analyzer ready!")

## 9. Test the Analyzer

In [ ]:
# Test Case 1: Obvious Scam
scam_text = """
Congratulations! You have been selected for a high-paying work from home job.
Salary: $10,000 per week guaranteed!
Just pay $500 registration fee to start immediately.
Contact us on WhatsApp: +1234567890
Send your bank account details for salary deposit.
"""

print("="*60)
print("TEST CASE 1: SCAM MESSAGE")
print("="*60)
result = analyzer.analyze_text(scam_text)

print(f"\n🔴 Trust Score: {result['trust_score']}/100")
print(f"📊 Status: {result['status_label']}")
print(f"📈 Scam Probability: {result['scam_probability']}%")
print(f"\n🚩 Red Flags Detected:")
for flag in result['red_flags']:
    print(f"   • {flag}")
print(f"\n💡 Advice:")
for advice in result['advice']:
    print(f"   • {advice}")

In [ ]:
# Test Case 2: Genuine Offer
genuine_text = """
Dear Candidate,

We are pleased to extend an offer of employment for the position of Software Engineer 
at TechCorp Inc. Your annual salary will be $95,000 with full benefits including 
health insurance and 401(k) matching.

Please review the attached offer letter and sign by Friday.
Your start date will be March 15, 2024.

Report to HR on your first day for onboarding.

Best regards,
HR Department
TechCorp Inc.
"""

print("="*60)
print("TEST CASE 2: GENUINE MESSAGE")
print("="*60)
result = analyzer.analyze_text(genuine_text)

print(f"\n🟢 Trust Score: {result['trust_score']}/100")
print(f"📊 Status: {result['status_label']}")
print(f"📈 Scam Probability: {result['scam_probability']}%")
print(f"\n🚩 Red Flags Detected:")
if result['red_flags']:
    for flag in result['red_flags']:
        print(f"   • {flag}")
else:
    print("   ✓ No red flags detected")
print(f"\n💡 Advice:")
for advice in result['advice']:
    print(f"   • {advice}")

In [ ]:
# Test Case 3: Suspicious (Caution)
suspicious_text = """
Hi, we saw your resume online and have an urgent opening!
The position is for a remote data entry specialist.
Salary is $3000 per month. No experience needed.
Reply on WhatsApp for more details.
"""

print("="*60)
print("TEST CASE 3: SUSPICIOUS MESSAGE")
print("="*60)
result = analyzer.analyze_text(suspicious_text)

print(f"\n🟡 Trust Score: {result['trust_score']}/100")
print(f"📊 Status: {result['status_label']}")
print(f"📈 Scam Probability: {result['scam_probability']}%")
print(f"\n🚩 Red Flags Detected:")
for flag in result['red_flags']:
    print(f"   • {flag}")
print(f"\n💡 Advice:")
for advice in result['advice']:
    print(f"   • {advice}")

## 10. Save the Model

In [ ]:
# Save all components for deployment
import pickle

model_package = {
    'model': best_model,
    'vectorizer': tfidf_vectorizer,
    'feature_names': feature_names
}

# Save as pickle
with open('trusthire_model.pkl', 'wb') as f:
    pickle.dump(model_package, f)

# Save individual components using joblib
joblib.dump(best_model, 'trusthire_rf_model.joblib')
joblib.dump(tfidf_vectorizer, 'trusthire_vectorizer.joblib')

print("✅ Model saved successfully!")
print("Files created:")
print("  - trusthire_model.pkl (complete package)")
print("  - trusthire_rf_model.joblib (Random Forest model)")
print("  - trusthire_vectorizer.joblib (TF-IDF vectorizer)")

## 11. Word Cloud Visualization

In [ ]:
# Generate word clouds for scam vs genuine
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scam word cloud
scam_text_combined = ' '.join(df[df['label'] == 1]['processed_text'])
wc_scam = WordCloud(width=800, height=400, background_color='white', 
                    colormap='Reds', max_words=100).generate(scam_text_combined)
axes[0].imshow(wc_scam, interpolation='bilinear')
axes[0].set_title('Scam Job Postings - Common Words', fontsize=14)
axes[0].axis('off')

# Genuine word cloud
genuine_text_combined = ' '.join(df[df['label'] == 0]['processed_text'])
wc_genuine = WordCloud(width=800, height=400, background_color='white', 
                       colormap='Greens', max_words=100).generate(genuine_text_combined)
axes[1].imshow(wc_genuine, interpolation='bilinear')
axes[1].set_title('Genuine Job Postings - Common Words', fontsize=14)
axes[1].axis('off')

plt.tight_layout()
plt.savefig('wordclouds.png', dpi=150, bbox_inches='tight')
plt.show()

## 12. API-Ready Function for Web Integration

In [ ]:
def analyze_job_offer(text: str) -> dict:
    """
    API-ready function to analyze a job offer.
    Can be used with Flask/FastAPI for web integration.
    
    Args:
        text: The job offer text to analyze
        
    Returns:
        dict: Analysis result with trust_score, status, findings, and advice
    """
    result = analyzer.analyze_text(text)
    
    # Format for API response
    return {
        'score': result['trust_score'],
        'status': result['status'],
        'findings': [
            {'type': 'danger' if 'Payment' in flag or 'Personal' in flag else 'warning', 
             'message': flag}
            for flag in result['red_flags']
        ],
        'advice': result['advice']
    }

# Test API function
test_result = analyze_job_offer("Pay $100 to get this amazing job opportunity!")
print("API Response Format:")
print(json.dumps(test_result, indent=2))

## Summary

This notebook implements a complete ML pipeline for TrustHire:

### Components Built:
1. **TextPreprocessor** - Cleans and lemmatizes text
2. **ScamFeatureExtractor** - Extracts 13+ scam-specific features
3. **Multiple ML Models** - Logistic Regression, Random Forest, SVM, etc.
4. **ImageOCR** - Extracts text from screenshots using Tesseract
5. **TrustHireAnalyzer** - Complete analysis pipeline

### Key Features:
- TF-IDF vectorization with bigrams
- Custom scam indicator detection
- Sentiment analysis
- OCR for image processing
- Trust score calculation (0-100)
- Red flag detection and explanation
- Smart safety advice generation

### Files Generated:
- `trusthire_model.pkl` - Complete model package
- `trusthire_rf_model.joblib` - Random Forest model
- `trusthire_vectorizer.joblib` - TF-IDF vectorizer
- `model_comparison.png` - Model performance chart
- `feature_importance.png` - Feature importance visualization
- `confusion_matrix.png` - Confusion matrix
- `wordclouds.png` - Word cloud comparison